## Desafío n.4 Vista de datos

In [2]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("SUPABASE_HOST")
DB_NAME = os.getenv("SUPABASE_DB")
DB_USER = os.getenv("SUPABASE_USER")
DB_PASS = os.getenv("SUPABASE_PASSWORD")
DB_PORT = os.getenv("SUPABASE_PORT", 5432)


conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    port=DB_PORT
)
conn.autocommit = True
cursor = conn.cursor()

print("Conectado a Supabase PostgreSQL")

create_view_sql = """
CREATE OR REPLACE VIEW vw_employees_summary AS
SELECT
    e.employee_id,
    e.first_name || ' ' || e.last_name AS full_name,
    e.email,
    d.department_name,
    j.job_title,
    e.salary,
    j.min_salary,
    j.max_salary,
    ROUND(AVG(e.salary) OVER (PARTITION BY j.job_id), 2) AS avg_salary_by_job,
    ROUND((e.salary - AVG(e.salary) OVER (PARTITION BY j.job_id)), 2) AS diff_vs_avg,
    ROUND(EXTRACT(YEAR FROM AGE(CURRENT_DATE, e.hire_date)), 1) AS years_in_company
FROM empleados e
JOIN departamentos d ON e.department_id = d.department_id
JOIN puestos j ON e.job_id = j.job_id;
"""

cursor.execute(create_view_sql)
print("Vista 'vw_employees_summary' creada o actualizada correctamente")

query = """
SELECT 
    department_name,
    job_title,
    ROUND(AVG(salary), 2) AS avg_salary,
    ROUND(MIN(salary), 2) AS min_salary,
    ROUND(MAX(salary), 2) AS max_salary,
    COUNT(*) AS num_employees
FROM vw_employees_summary
GROUP BY department_name, job_title
ORDER BY department_name, avg_salary DESC;
"""

df_report = pd.read_sql(query, conn)

print("\nVista analítica de empleados:\n")
print(df_report.head(10))

cursor.close()
conn.close()
print("\nConexión cerrada correctamente.")


Conectado a Supabase PostgreSQL
Vista 'vw_employees_summary' creada o actualizada correctamente

Vista analítica de empleados:

  department_name  job_title  avg_salary  min_salary  max_salary  \
0        Finanzas   Director  6018379.14   3357642.0   9474988.0   
1        Finanzas   Analista  5854594.35   4718508.0   6859106.0   
2        Finanzas    Gerente  4408009.15   2589437.0   6068235.0   
3        Finanzas    Técnico  4392431.05   3049247.0   5714921.0   
4        Finanzas  Asistente  3774564.26   2119229.0   5850127.0   
5              IT   Director  6255174.48   3229655.0   9516985.0   
6              IT   Analista  5831244.89   4702250.0   6813607.0   
7              IT    Gerente  4252845.05   2500812.0   5983299.0   
8              IT    Técnico  4055655.47   3071561.0   5666085.0   
9              IT  Asistente  4011990.15   2331886.0   5609607.0   

   num_employees  
0             36  
1             37  
2             34  
3             56  
4             39  
5        

C:\Users\carlo\AppData\Local\Temp\ipykernel_21296\4268851966.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_report = pd.read_sql(query, conn)
